## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

## Get the dataset

In [2]:
data = pd.read_csv('./../data/regions_cleaned_with_diff_shifted_cci.csv')

### The different between cleaning last time in days

In [3]:
completed_ids = set()
for index, row in data.iterrows():
    my_tuple = (str(row['osm_id']),str(row['cci_id']))
    if my_tuple not in completed_ids:
        completed_ids.add(my_tuple)

In [20]:
import dateutil
list_dataframes = []
count = 1
## Iterate over the completed ids
for completed_id in completed_ids: 
    
    temp_df = data[data['osm_id'].astype(str).str.contains(completed_id[0])]
    data_f = temp_df[temp_df['cci_id'].astype(str).str.contains(completed_id[1])]
    
    if not data_f.empty:
        pass
        # data_f = data_f.sort_values(by='date')
    if count > 38:
        break
    count+=1

In [21]:
# my_features = ['day_type','event','diff_nearest_clean','mean_temp','total_prep']
my_features = ['day_type','mean_temp','total_prep']
my_features_df = data_f[my_features].copy(deep=True)
print(my_features_df.shape)

test_feature = 'cci'
test_feature_df = data_f[test_feature].copy(deep=True)

(2, 3)


In [22]:
combined_df = pd.concat([my_features_df, test_feature_df], axis=1, ignore_index=False)
# combined_df.tail()

In [23]:
combined_df = combined_df.fillna(7.0).copy()

In [24]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

train_data, test_data = train_test_split(combined_df, test_size=0.1)

In [17]:
train_X = train_data[my_features]
# train_X = preprocessing.scale(train_X)
train_X = preprocessing.MinMaxScaler().fit_transform(train_X)

train_Y = train_data[test_feature]

test_X = test_data[my_features]
# test_X = preprocessing.scale(test_X)
test_X = preprocessing.MinMaxScaler().fit_transform(test_X)


test_Y = test_data[test_feature]

In [18]:
# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_X, train_Y)

# Make predictions using the testing set
test_pred = regr.predict(test_X)

In [19]:
# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(test_Y, test_pred))
# Explained variance score: 1 is perfect prediction
from sklearn.metrics import explained_variance_score
print('Variance score: %.2f' % explained_variance_score(test_Y, test_pred))

# Plot outputs
# plt.scatter(test_data[['day_type','months']], test_data['cci'],  color='black')
# plt.plot(test_data[['day_type','months']], test_pred, color='blue', linewidth=3)

# plt.xticks(())
# plt.yticks(())

# plt.show()

Coefficients: 
 [ 0.28175711 -0.08354065 -0.70675542]
Mean squared error: 0.32
Variance score: -0.87
